<a href="https://colab.research.google.com/github/SugarGuan/Learn-ML-DL/blob/master/Tensorflow_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

W0630 00:45:22.660529 139951775647616 deprecation.py:323] From <ipython-input-2-5839acd4638a>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0630 00:45:22.662754 139951775647616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0630 00:45:22.667889 139951775647616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz


W0630 00:45:23.435514 139951775647616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0630 00:45:23.443677 139951775647616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0630 00:45:23.753119 139951775647616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
# Create a tensorflow sesion
# Have to close the session when you want to re-run any part of this ipynb file!
sess = tf.InteractiveSession()

In [0]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return initial

In [0]:
# 卷积层
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

# 池化层（使用max 最大特征提取方式进行池化）
def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')


In [0]:
# 定义输入
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

x_image = tf.reshape(x, [-1,28,28,1])

In [0]:
# 定义卷积层1
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+ b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [0]:
# 定义卷积层2
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [0]:
# 全连接层
W_fc1 = weight_variable([7 * 7 * 64 , 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [11]:
# Dropout层 通过placeholder传入一个keep_prob比率进行控制
# 训练中随机丢弃一些数据 来减轻过拟合（如果保留全部数据可以获得最好性能）
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W0630 00:46:17.417896 139951775647616 deprecation.py:506] From <ipython-input-11-35c3bf3a9559>:2: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# Softmax 层，用于得到最后的概率输出
W_fc2 = weight_variable([1024 , 10])
b_fc2 = weight_variable([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [0]:
# 定义损失函数 使用交叉熵（cross entropy）
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv),
                              reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [0]:
# 评测准确率

In [0]:
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_ ,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
# 初始化全局变量（variables）
tf.global_variables_initializer().run()
for i in range(20000):
  batch = mnist.train.next_batch(50)
  if i % 100 == 0:
    train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_:batch[1], 
                                             keep_prob: 1.0})
    print("Step %d , training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_:batch[1], keep_prob: 0.5})

Step 0 , training accuracy 0.18
Step 100 , training accuracy 0.88
Step 200 , training accuracy 0.86
Step 300 , training accuracy 0.94
Step 400 , training accuracy 0.96
Step 500 , training accuracy 0.96
Step 600 , training accuracy 0.94
Step 700 , training accuracy 0.96
Step 800 , training accuracy 0.92
Step 900 , training accuracy 1
Step 1000 , training accuracy 0.98
Step 1100 , training accuracy 0.98
Step 1200 , training accuracy 0.96
Step 1300 , training accuracy 0.98
Step 1400 , training accuracy 0.98
Step 1500 , training accuracy 0.94
Step 1600 , training accuracy 1
Step 1700 , training accuracy 1
Step 1800 , training accuracy 0.92
Step 1900 , training accuracy 0.96
Step 2000 , training accuracy 1
Step 2100 , training accuracy 1
Step 2200 , training accuracy 0.98
Step 2300 , training accuracy 0.96
Step 2400 , training accuracy 1
Step 2500 , training accuracy 0.98
Step 2600 , training accuracy 0.98
Step 2700 , training accuracy 0.98
Step 2800 , training accuracy 1
Step 2900 , traini

In [16]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_:mnist.test.labels, keep_prob: 1.0}))

test accuracy 0.9915


In [0]:
 # tf.InteractiveSession.close()